### Cylindrical cells 

In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib qt


In [2]:
### Typical number of cells around the cylinder
n_circum = 8
### Typical number of cells along z
n_length = 7
### Distance between 2 cells
l_0 = 1. 
### Cells heights
h_0 = 1.

In [27]:
rho_c = n_circum * l_0 / (2 * np.pi)
rho_lumen = rho_c - h_0

delta_theta = 2 * np.pi / n_circum
delta_z = delta_theta * rho_c * np.sqrt(3)/2.

zt_grid = np.mgrid[:n_length, :n_circum]
thetas = zt_grid[1].astype('float')
thetas[::2, ...] += 0.5
thetas *= delta_theta
thetas = thetas.flatten()
zeds = zt_grid[0].astype('float')
zeds *= delta_z
zeds -= zeds.max() / 2
zeds = zeds.flatten()

pos = np.zeros((n_circum * n_length, 3))

pos[:, 0] = np.cos(thetas) * rho_c
pos[:, 1] = np.sin(thetas) * rho_c
pos[:, 2] = zeds

In [28]:
fig, axes = plt.subplots(2, 2, sharex=True)


axes[0, 0].plot(pos[:, 0], pos[:, 1], 'o')
axes[0, 0].set_aspect('equal')


axes[0, 1].plot(pos[:, 2], pos[:, 0], 'o')
axes[0, 1].set_aspect('equal')

axes[1, 0].plot(pos[:, 1], pos[:, 2], 'o')
axes[1, 0].set_aspect('equal')

axes[1, 1].plot(thetas * rho_c, pos[:, 2], 'o')
axes[1, 1].set_aspect('equal')


In [13]:
import graph_tool.all as gt

In [22]:
l_0

1.0

In [30]:
cells_graph, pos_vp = gt.geometric_graph(pos, l_0*1.1)

In [45]:
rhos = np.linalg.norm(pos[:, :2], axis=1)
z_angle = np.pi/24
d_theta = np.pi/12

pseudo_x = pos[:, 2] * np.cos(z_angle) - rhos * np.sin(thetas + d_theta) * np.sin(z_angle)
pseudo_y = rhos * np.cos(thetas + d_theta)

pseudo_pos = gt.graph_draw(cells_graph, output='generated_cells.png')

![the generated cells](generated_cells.png)

In [52]:
import pandas as pd
pos = pd.DataFrame(pos, columns=['x', 'y', 'z'])


In [54]:
pos.index.name = 'cell'

pos.head()


,x,y,z
cell,,,
0,1.176320,0.487248,-2.598076
1,0.487248,1.176320,-2.598076
2,-0.487248,1.176320,-2.598076
3,-1.176320,0.487248,-2.598076
4,-1.176320,-0.487248,-2.598076


In [59]:
from scipy.spatial import Voronoi, voronoi_plot_2d

In [75]:
plt.plot(pos.rhos, pos.sigmas, 'o')

In [ ]:
thetas = np.arctan2(pos['y'], pos['x'])
rhos = np.hypot(pos['y'], pos['x'])
pos['sigmas'] = thetas * rhos
pos['rhos'] = rhos


voronoi = Voronoi(pos[['z', 'sigmas']].values)
#plt.plot(pos.z, pos.sigmas, 'bo')
fig = voronoi_plot_2d(voronoi)
ax = plt.gca()

ax.set_aspect('equal')

In [ ]:

pos['sigmas'] = thetas * rhos
pos['rhos'] = rhos


voronoi = Voronoi(pos[['z', 'sigmas']].values)
#plt.plot(pos.z, pos.sigmas, 'bo')
fig = voronoi_plot_2d(voronoi)
ax = plt.gca()

ax.set_aspect('equal')
